# Setup 

In [2]:
# Imports and Environmental Setup

import sys
sys.path.insert(0, '..')

import warnings
from tqdm import TqdmExperimentalWarning
warnings.simplefilter('ignore', TqdmExperimentalWarning)

import matplotlib
%matplotlib inline

import ipywidgets 
import agglomerative

style = {'description_width': 'initial'}

from PIL import Image, ImageChops

# https://mail.python.org/pipermail/python-list/2003-October/226212.html
def autocrop(im, bgcolor):
    if im.mode != "RGB":
        im = im.convert("RGB")
    bg = Image.new("RGB", im.size, bgcolor)
    diff = ImageChops.difference(im, bg)
    bbox = diff.getbbox()
    if bbox:
        return im.crop(bbox)
    return None # no contents

# Settings 

## Types of Settings to change:

### Algorithm
1. **No Preclustering**
2. **PC1**: Pre Cluster + Cell-level Intra-Cluster + cell-level/Column-level inter-cluster edge selection
3. **PC2**: Pre cluster + cell-level & Column level intra-cluster + Cell-level & Column-level inter-cluster edge selection

### Thresholds
1. Intra-Cluster Cell-Level Jaccard Edge Inclusion Threshold
2. Inter-Cluster Cell-Level Jaccard Edge Inclusion Threshold

In [3]:
def run_relic(notebook, algorithm='PC2',intra_cell_threshold=0.01,inter_cell_threshold=0.01):
    global result, im_frames
    result, im_frames = agglomerative.lineage_inference_agglomerative(nb_name=notebook, 
                                                                      pre_cluster=algorithm,
                                                                      intra_cell_threshold=intra_cell_threshold,
                                                                      inter_cell_threshold=inter_cell_threshold)
    display(result)
    display(autocrop(im_frames[-1], 'white'))


im = ipywidgets.interact_manual(run_relic, notebook = agglomerative.notebooks,
    algorithm = ['No Precluster', 'PC1', 'PC2'],
    intra_cell_threshold = (0.0,1.0,0.001),
    inter_cell_threshold = (0.0,1.0,0.001))



interactive(children=(Dropdown(description='notebook', options=('20200211-145628', '20200211-145648', '2020021…

# Step-By-Step Analysis 

In [4]:
# Analyze the results
l = len(result.index)

def display_result(step=0):
    display(result.loc[[step]][['numclusters', 'edges_correct', 'edges_missing', 'edges_to_remove', 'F1']])
    image = im_frames[step]
    display(autocrop(image,'white'))
    
plot = ipywidgets.interactive(display_result, step=(0,l-1))    
output = plot.children[-1]
output.layout.height = str(autocrop(im_frames[-1],'white').height - 600)+'px'

display(plot)

interactive(children=(IntSlider(value=0, description='step', max=8), Output(layout=Layout(height='968px'))), _…